# Statistics

In the [drawing graphs](./2-drawing-graphs.ipynb) notebook, when plotting record start and end dates for different ships, the records started to show clusters around times when the Royal Navy used a ship name. This notebook explores that further. 

In [ ]:
%pip install -q scikit-learn
%pip install -q csv
%pip install -q matplotlib
%pip install -q numpy
%pip install -q scipy
%pip install -q pandas

import scipy.cluster.hierarchy as sch
from scipy.cluster.hierarchy import fcluster
from scipy import stats

import pandas

import numpy as np

import matplotlib.pyplot as plt

from sklearn.decomposition import PCA

## Data

This notebook will use the same dataset as the drawing graphs one.

In [ ]:
data = pandas.read_csv('ship_data.csv', sep=',', header=0)
dataframe = pandas.DataFrame(data)
print(dataframe.head())

This time, however, a few more modifications to the data are needed. The date-based columns are again converted to datetime objects, and the record duration is calculated again. As the statistical analysis run here will all only work with numerical data, some of the columns are dropped, and others are converted to numerical data.

In [ ]:
numberic_columns = dataframe[['startDate', 'endDate', 'record_duration', 'reference', 'description_length', 'ship']]

numberic_columns['reference'] = dataframe['reference'].str.split(' ').str[1]
numberic_columns['reference'] = pandas.to_numeric(numberic_columns['reference'].str.split('/').str[0])

numberic_columns['startDate'] = pandas.to_datetime(dataframe['startDate']).values.astype(float)
numberic_columns['endDate'] = pandas.to_datetime(dataframe['endDate']).values.astype(float)

durations = []
for duration in dataframe['record_duration']:
    str(durations.append(duration.split(" ")[0]))

numberic_columns['record_duration'] = durations

print(numberic_columns.head())

For convenience, a subset of the data (in this case, records to do with "Boxer") is isolated and made available. This smaller dataset will make it easier to read some of the example graphs and tables.

In [ ]:
boxer_data = numberic_columns[numberic_columns['ship'] == 'Boxer']
boxer_data = boxer_data.drop(columns=['ship'])
print(boxer_data.head())

## Dendrogram/Hierarchical Clustering

To investigate whether the data are actually forming clusters, we can use a hierarchical clustering approach, and plot it on a dendrogram. 

This analysis looks at the data its given, and groups similar data together. For example, if two records started in 1850 and 1851, and the third in 1910, then the first two would be grouped together. This is comparison is then done across all records, using all the columns. This analysis is then repeated at different levels of similarity; allowing greater variation in the data within a cluster - for example, grouping a record that starts in 1850 with one that starts in 1870. This is repeated until all records are in one cluster. 

The results are then plotted on a dendrogram, shown in the next cell. In this example, the `ward` distance calculation method is used, which attempts to minimise the variance of clusters being joined. Dendrograms should be read similarly to family trees; the individual records are shown on the x axis at the bottom of the graph, with the branches representing the different clusters. The longer the branch, the more statistically significant the cluster. 

In [ ]:
plt_dendro, ax_dendro = plt.subplots()
dendrogram = sch.dendrogram(sch.linkage(boxer_data, method='ward'), ax=ax_dendro)
plt.show()

Working up from the base of the dendrogram, we can see that the individual records quickly cluster into three distinct clusters, matching the expected outcome from the graph. The top half of the graph shows there may be just two clusters, which must be kept in consideration. 

## Using these clusters

Information about the number of clusters allows further analysis and visualisation of the clusters. 

The first step is assign each record to a cluster. The [`fcluster`](https://pythonguides.com/python-scipy-fcluster/) method from scipy uses the same hierarchical clustering method from the dendrogram, but here, the number of clusters is pre-specified. The output is a list, with the cluster number for each record, in order. 

In [ ]:
wardlabel = fcluster(sch.linkage(boxer_data, method='ward'), 3, criterion='maxclust')

print(wardlabel)

We can then use this list of cluster numbers to colour records in scatter plots, for example. Here, we are drawing a pair, both with start date on the x-axis, one plotted against end date, and the other against duration. 

In [ ]:
fig_cluster_scatter, (ax_cluster_scatter_1, ax_cluster_scatter_2) = plt.subplots(1, 2, figsize=(20, 10))
ax_cluster_scatter_1.scatter(boxer_data['startDate'], boxer_data['endDate'], c=wardlabel, cmap='viridis')
ax_cluster_scatter_1.set_xlabel('Start Date')
ax_cluster_scatter_1.set_ylabel('End Date')
ax_cluster_scatter_1.set_title('Start Date vs End Date')
ax_cluster_scatter_2.scatter(boxer_data['startDate'], boxer_data['record_duration'], c=wardlabel, cmap='viridis')
ax_cluster_scatter_2.set_xlabel('Start Date')
ax_cluster_scatter_2.set_ylabel('Record Duration')
ax_cluster_scatter_2.set_title('Start Date vs Record Duration')
plt.show()

Where before, plotting this data may have shown the 3 clusters, but might have not been clear. With the clusters coloured, the clusters the records belong to becomes much clearer. If the clusters are close together, the colour can be particularly useful in distinguishing between them.

## Linear regression

The start date vs end date graph for the Boxer data looks very similar to a straight line; a linear regression tests how closely a straight line compares to the data. This test provides you with a value for the slope of the line, and the y-intercept, giving enough information to plot the line. It also provides an r-value and p-value. The p-value represents the probability that the slope of the line is 0. The r-value is the Pearson correlation coefficient, which, when squared, becomes the coefficient of determination. This value represents the proportion of the variance in the dependent variable that is predictable from the independent variable - testing to see if the line predicts the data well.

This test is available via SciPy, and compares two lists of equal length. With a small amount of extra work, you can then plot the line onto the scatter graph. This next cell does all of these steps. 

In [ ]:
x = []
y = []

for index, row in boxer_data.iterrows():
    x.append(row['startDate'])
    y.append(row['endDate'])

regression = stats.linregress(x, y)
print(regression)

print("R squared: " + str(regression.rvalue ** 2))

plt_regression, ax_regression = plt.subplots()
ax_regression.scatter(x, y, c=wardlabel, cmap='viridis')
line_y = [regression.slope * i + regression.intercept for i in x]
ax_regression.plot(x, line_y, color='red')
ax_regression.set_xlabel('Start Date')
ax_regression.set_ylabel('End Date')
ax_regression.set_title('Start Date vs End Date')
plt.show()

The R-squared value for the Boxer data ends up being 0.999, very close to 1, indicative of a very strong linear relationship. Note that this might be somewhat expected - the graph is start date vs end date, so the two are directly related.

## All the data 

Rather than drawing scatter diagrams for each combination of variables, it is possible to draw a single scatter plot of all the data at once, while keeping as much of the information as possible. This is achieved through a scatter diagram based on a Principal Component Analysis (PCA).

Plotting a PCA is similar to plotting a 2-d scatter plot, but taking into account multiple dimensions. Imagine a 3-d scatter plot, where the points all formed a perfect sphere - plotting a silhouette in two dimensions, the graph would look like a circle. A PCA achieves this with as many dimensions as supplied (with dimensions here referring to columns in the data, or variables). PCAs aim to show as much variation in the data as possible while doing this: for example, if 3d-scatter looked like a rugby ball, then the projection would look more like an oval. To make PCAs easier to read, variables from the original data can be drawn onto the scatter plot as arrows - with longer arrows generally representing a stronger effects. 

The 'components' in the name of the analysis relates to these factors used to project the data. A PCA aims to reduce the number of dimensions (or columns), while keeping as much variation as possible. Thus, if two dimensions are very similar, the PCA keep them as a single component.

The first step is another tweak of the data - where previously we were working with the numeric only data relating to Boxer, we need a numeric only dataset for all ships. 

In [ ]:
numberic_columns = numberic_columns.drop(columns=['ship'])

print(numberic_columns.head())

The first step in plotting the PCA is to calculate the PCA itself. The components are then made available. The loadings (known as such as they show how much the variable loads onto the component) are calculated using this information. 

In [ ]:
pca = PCA()
components = pca.fit_transform(numberic_columns)
loadings = pca.components_.T * np.sqrt(pca.explained_variance_)

These components are then used to plot the PCA. The scatter plot is drawn, and the loadings are drawn as arrows.

In [ ]:
fig_pca, ax_pca = plt.subplots(figsize=(15, 15))
ax_pca.scatter(components[:, 0], components[:, 1], c=numberic_columns['reference'], cmap='viridis')
for i, (x, y) in enumerate(zip(loadings[:, 0], loadings[:, 1])):
    ax_pca.arrow(0, 0, x, y, color='r', alpha=0.5)
    ax_pca.text(x, y, numberic_columns.columns[i], fontsize='12', ha='right')
ax_pca.set_xlabel('PC1')
ax_pca.set_ylabel('PC2')
ax_pca.set_title('PCA')
plt.show()

Different components in the PCA will have stronger or weaker effects on the outcome, and some may have so little effect that they are statistically insignificant. A scree plot shows their different levels with the components shown on the x-axis and level of effect they have on the y-axis. Higher numbered components have less effect on the PCA, leading to the slope of the graph, and the name 'scree'-plot - they look like the scree slope on a mountain. 

There are two ways to read a scree plot: 
- Look for an 'elbow' - typically, a scree plot will have a point where it goes from a steep slope to a shallow one. To determine which factors to keep, look for this point, and keep all factors to the left. 
- Take all factors with an effect of 1 or above. 

In [ ]:
pc_values = np.arange(pca.n_components_) + 1
fig_pca_variance, ax_pca_variance = plt.subplots()
ax_pca_variance.plot(pc_values, pca.explained_variance_ratio_, 'ro-', linewidth=2)
ax_pca_variance.set_title('Scree Plot')
ax_pca_variance.set_xlabel('Principal Component')
ax_pca_variance.set_ylabel('Proportion of Variance Explained')
plt.show()

From this scree plot, we can see that there really is only 1 component causing any variation. In this example, this is almost certainly a result of the artificial nature of the data - we have chosen to keep start and end date, and have calculated record duration from these. These values are almost certainly going to be working together as the only component. With more complex datasets, scree plots can become more useful, variable, and require more careful consideration.

# Further steps and conclusion

This notebook only scratches the surface of the statistical analysis that can be run, and does not go into much detail on the methods used. Depending on what has interested you about this notebook, there are a few different directions you could go in:

- If the possibility of grouping together records in clusters has interested you, there are other methods of clustering, with different strengths and weaknesses depending on what data are available.

- If you have a research question but the data has missing values, there are techniques to fill in these gaps, and test whether this has affected the results.

- If being able to analyse large datasets has opened new research questions for you to investigate, there are various statistical methods available, simply needing the right question to be asked.